In [129]:
import pandas as pd
import random
import requests
import json

g_api = ""

#Importing Raw IRS Data
irs_pd = pd.read_csv("Raw Data/IRS Dataset by County.csv")
irs_pd.head(10)


,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,Alabama,1,25120.0,14000.0,9140.0,1110.0,17260.0,...,4530.0,9068.0,70.0,97.0,0.0,0.0,2750.0,3791.0,10210.0,35026.0
1,1,AL,0,Alabama,2,300920.0,223950.0,26770.0,46400.0,156430.0,...,77720.0,45164.0,0.0,0.0,0.0,0.0,23290.0,10777.0,250850.0,296608.0
2,1,AL,0,Alabama,3,549200.0,257890.0,88300.0,194060.0,323470.0,...,312340.0,280080.0,0.0,0.0,0.0,0.0,39900.0,37545.0,494200.0,1668817.0
3,1,AL,0,Alabama,4,491350.0,200730.0,150260.0,125560.0,281320.0,...,397110.0,950375.0,30.0,20.0,0.0,0.0,70960.0,102612.0,413810.0,1177609.0
4,1,AL,0,Alabama,5,259320.0,75790.0,142750.0,34060.0,156510.0,...,250010.0,1316842.0,0.0,0.0,0.0,0.0,62330.0,133274.0,191980.0,538147.0
5,1,AL,0,Alabama,6,164700.0,26720.0,125180.0,10400.0,99550.0,...,163440.0,1389769.0,0.0,0.0,0.0,0.0,45250.0,125396.0,115550.0,376309.0
6,1,AL,0,Alabama,7,202400.0,18820.0,175890.0,5840.0,121460.0,...,201840.0,3541123.0,850.0,240.0,440.0,166.0,80640.0,371780.0,114450.0,449311.0
7,1,AL,0,Alabama,8,53390.0,4380.0,47580.0,950.0,40960.0,...,53350.0,5779823.0,20860.0,36718.0,27690.0,104622.0,29700.0,863579.0,16300.0,290677.0
8,1,AL,1,Autauga County,1,210.0,120.0,80.0,0.0,140.0,...,30.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,308.0
9,1,AL,1,Autauga County,2,3350.0,2650.0,230.0,430.0,1320.0,...,770.0,372.0,0.0,0.0,0.0,0.0,230.0,99.0,2880.0,2886.0


In [130]:
# #Removing state-level data
irs_pd = irs_pd[irs_pd.COUNTYFIPS !=0]

#Combining State and County columns to more easily merge with CDC data
irs_pd["COUNTYNAME"] = irs_pd["COUNTYNAME"] + ", " + irs_pd["STATE"]

#Removing Rows with Missing Data
irs_pd = irs_pd.dropna(how='any')

#Pulling Needed Information
irs_basic = irs_pd[["COUNTYNAME", "agi_stub", "N1", "A00200"]]
irs_basic.head(10)


,COUNTYNAME,agi_stub,N1,A00200
8,"Autauga County, AL",1,210.0,1403.0
9,"Autauga County, AL",2,3350.0,14437.0
10,"Autauga County, AL",3,5620.0,77486.0
11,"Autauga County, AL",4,5410.0,159789.0
12,"Autauga County, AL",5,3420.0,163864.0
13,"Autauga County, AL",6,2450.0,163709.0
14,"Autauga County, AL",7,3000.0,302302.0
15,"Autauga County, AL",8,360.0,61298.0
16,"Baldwin County, AL",1,1710.0,13073.0
17,"Baldwin County, AL",2,13400.0,53151.0


In [131]:
#Renaming Columns
irs_rename = irs_basic.rename(columns={"COUNTYNAME":"County",
                                       "agi_stub":"Adjusted Gross Income",
                                       "N1":"Number of Returns",
                                       "A00200":"Total Salaries & Wages"})

irs_rename.head(10)


,County,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
8,"Autauga County, AL",1,210.0,1403.0
9,"Autauga County, AL",2,3350.0,14437.0
10,"Autauga County, AL",3,5620.0,77486.0
11,"Autauga County, AL",4,5410.0,159789.0
12,"Autauga County, AL",5,3420.0,163864.0
13,"Autauga County, AL",6,2450.0,163709.0
14,"Autauga County, AL",7,3000.0,302302.0
15,"Autauga County, AL",8,360.0,61298.0
16,"Baldwin County, AL",1,1710.0,13073.0
17,"Baldwin County, AL",2,13400.0,53151.0


In [132]:
#Cleaning Adjusted Gross Income
irs_rename.loc[:, "Adjusted Gross Income"] = irs_rename["Adjusted Gross Income"].astype("str")

irs_rename['Adjusted Gross Income'] = irs_rename['Adjusted Gross Income'].replace(
    {'0': 'No AGI Stub',
     '1': 'Under $1',
     '2': '1 - $9,999',
     '3': '10,000 - $24,999',
     '4': '25,000 - $49,999',
     '5': '50,000 - $74,999',
     '6': '75,000 - $99,999',
     '7': '100,000 - $199,999',
     '8': '$200,000 or more'
    })

irs_rename.head(10)


,County,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
8,"Autauga County, AL",Under $1,210.0,1403.0
9,"Autauga County, AL","1 - $9,999",3350.0,14437.0
10,"Autauga County, AL","10,000 - $24,999",5620.0,77486.0
11,"Autauga County, AL","25,000 - $49,999",5410.0,159789.0
12,"Autauga County, AL","50,000 - $74,999",3420.0,163864.0
13,"Autauga County, AL","75,000 - $99,999",2450.0,163709.0
14,"Autauga County, AL","100,000 - $199,999",3000.0,302302.0
15,"Autauga County, AL","$200,000 or more",360.0,61298.0
16,"Baldwin County, AL",Under $1,1710.0,13073.0
17,"Baldwin County, AL","1 - $9,999",13400.0,53151.0


In [133]:
#Importing Raw CDC Data
cdc_pd = pd.read_csv("Raw Data/CDC Dataset - Heart Disease by County.csv")
cdc_pd.head()

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,TopicID,LocationID,Location 1
0,2014,AK,Aleutians East,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,105.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2013,"(55.440626, -161.962562)"
1,2014,AK,Aleutians West,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,211.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2016,"(52.995403, -170.251538)"
2,2014,AK,Anchorage,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,257.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2020,"(61.159049, -149.103905)"
3,2014,AK,Bethel,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,351.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2050,"(60.924483, -159.749655)"
4,2014,AK,Bristol Bay,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,NaN,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",~,Insufficient Data,Gender,Overall,Race/Ethnicity,Overall,T2,2060,"(58.754192, -156.694709)"


In [134]:
#Removing state-level data
cdc_pd = cdc_pd[cdc_pd.GeographicLevel !="State"]

#Removing Insufficient Data
cdc_pd = cdc_pd[cdc_pd.Data_Value_Footnote_Symbol !="~"]

#Combining State and County columns to more easily merge with IRS data
cdc_pd["LocationDesc"] = cdc_pd["LocationDesc"] + ", " + cdc_pd["LocationAbbr"]

#Pulling Needed Information
cdc_basic = cdc_pd[["LocationAbbr",
                    "LocationDesc",
                    "Data_Value",
                    "Stratification1",
                    "Stratification2",
                    "Location 1"]]
cdc_basic.head()

,LocationAbbr,LocationDesc,Data_Value,Stratification1,Stratification2,Location 1
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"


In [135]:
#Renaming Columns
cdc_rename = cdc_basic.rename(columns={"LocationAbbr":"State",
                                       "LocationDesc":"County",
                                       "Data_Value":"Deaths per 100,000",
                                       "Stratification1":"Gender",
                                       "Stratification2":"Race/Ethnicity",
                                       "Location 1": "Geographical Coordinates"})

cdc_rename.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"
6,AK,"Dillingham, AK",411.6,Overall,Overall,"(59.803151, -158.181608)"
7,AK,"Fairbanks North Star, AK",305.7,Overall,Overall,"(64.809327, -146.586265)"
8,AK,"Haines, AK",295.7,Overall,Overall,"(59.140721, -135.513878)"
9,AK,"Juneau, AK",295.7,Overall,Overall,"(58.478434, -134.150437)"
10,AK,"Kenai Peninsula, AK",299.4,Overall,Overall,"(60.271254, -151.556972)"


In [136]:
#Dropping NaN Rows
cdc_rename = cdc_rename.dropna(how='any')
cdc_rename.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"
6,AK,"Dillingham, AK",411.6,Overall,Overall,"(59.803151, -158.181608)"
7,AK,"Fairbanks North Star, AK",305.7,Overall,Overall,"(64.809327, -146.586265)"
8,AK,"Haines, AK",295.7,Overall,Overall,"(59.140721, -135.513878)"
9,AK,"Juneau, AK",295.7,Overall,Overall,"(58.478434, -134.150437)"
10,AK,"Kenai Peninsula, AK",299.4,Overall,Overall,"(60.271254, -151.556972)"


In [137]:
df_merge = pd.merge(cdc_rename, irs_rename, on="County")
df_merge.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
0,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)",Under $1,100.0,0.0
1,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","1 - $9,999",1160.0,4069.0
2,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","10,000 - $24,999",1270.0,15724.0
3,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","25,000 - $49,999",1680.0,48739.0
4,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","50,000 - $74,999",1080.0,50947.0
5,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","75,000 - $99,999",690.0,42393.0
6,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","100,000 - $199,999",1050.0,98756.0
7,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","$200,000 or more",220.0,27889.0
8,AK,"Ketchikan Gateway, AK",496.9,Male,Overall,"(55.559161, -131.351892)",Under $1,100.0,0.0
9,AK,"Ketchikan Gateway, AK",496.9,Male,Overall,"(55.559161, -131.351892)","1 - $9,999",1160.0,4069.0


In [138]:
df_merge.to_csv("Clean_Data_Merged.csv", index=False, header=True)

PermissionError: [Errno 13] Permission denied: 'Clean_Data_Merged.csv'

In [ ]:
unique_counties = df_merge["County"].unique()
len(unique_counties)

In [ ]:
random.shuffle(unique_counties)
county_pull = unique_counties[:500]
county_pull

In [156]:
# geocoordinates
#target_search = "Chinese"

target_type = "restaurant"
county = county_pull[0]
types = ["point_of_interest",
         "restaurant",
         "food"]

# set up a parameters dictionary
params = {
    "location": "43.6187102, -116.2146068",
    "name": "Burger",
    "radius": 50000,
    "type": "restaurant",
    "key": g_api
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAA4M66K_9cBUBkBQzUU4T1k8P3qqcJGZAVlm7kpVxGVXEEzNYMQkDpctoVB4RiIwdNCRnWOakch-V2yQs_ARdNd6kp5jw0-j_GeVaw2nHa69kQiUi0RsLRpeJQQ7ykDVDHJvBbR37_zIkioPwEBRMxHqsqZqbeJmgZtjhvRdU-AL9f-JONAG6uZPpmIbrny5VPRe4cZ9RMELaNcD7Q883Hfw7FhRO68wMHlRIxZVbqB5PT9oC156b3IKgvPgr_Mr89O2XBn2Ec-rtbtj1IycMcRuukp51diwEVvkG1x8MTRMy39liHZG_QCJ42hf3N-SQQSVsjMAQW6qHBRe-x8y0M5Dv5eyj4K8VDuqShsrAEx3hz0zpbK-GvvOIiwDzRYnS9kzv--jBaRtj3g4Kv7QZvcSEIlo2EToYBdH7uivBRUVPOYaFEB0nYdgHOTgBJdc5W_LbN3oW41E",
    "results": [
        {
            "formatted_address": "224 N 10th St, Boise, ID 83702",
            "geometry": {
                "location": {
                    "lat": 43.6179926,
                    "lng": -116.2044116
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61939097989272,
                        "lng": -116.2031573701073
                    },
                    "southwest": {
   